# Data Versioning

Keeping track of versions of your data can be tricky. Historically, data scientists might simply keep copies of all versions of data that they use, and rename them appropriately, which can clutter up your data store or file system, and makes it difficult to truly keep track of which data went with which model (experiment). 

The notion of having a data version control system, similar to what git does for code, has been around for awhile, and the most popular tools that have emerged to address these needs have been Git LFS and DVC. But, some would argue that using a version control tool for data is unnecessary, so long as the data scientist is disciplined enough to create reproducible data pipelines themselves, and so long as the data sources themselves are managed correctly. Regardless, I think it's useful to give data versioning a try, to understand its limitations.

## Data Version Control (DVC)

[DVC](https://dvc.org/) is a tool that can be used to track versions of data, and is used similarly to git. First, let's install DVC using 

`pip install dvc`  
`pip install dvc-s3`

from within your environment. There is no need to add this to your requirement.txt file, we are using it only for demonstration purposes. After it's installed you can check the version and help files:

`dvc -V`  
`dvc -h`

Then, in your project root folder you can initialize DVC, similar to initializing a git repo, using 

`dvc init` 

From here, you can see that there is a new .dvc folder and a .dvcignore file. The .dvcignore file is similar to .gitignore, you use it to tell DVC if there are certain data files you want DVC to ignore. The .dvc folder contains a config file and a tmp folder, and should also have a .gitignore file. This folder will eventually have a cache folder in it as well. We will talk about this in a minute. First, let's add the .dvc/config file and .dvcignore file to git. 

`git add .dvc/config .dvcignore .dvc/.gitignore`  or `git add .` might work if you don't have a bunch of other uncommited changes.  
`git commit -m 'add dvc'`  

You may want to push these to your Github repo at this point.

`git push origin main`  

Now you can start version-controlling data. Let's add a dataset to our data/ folder. You may already have data there, but you can also use any toy dataset that you want to use for practicing with DVC. I will add the adult.data file into my data/ folder and then add it to DVC (again, similar to git) using

`dvc add data/adult.data`

I can now see an adult.data.dvc file in the data/ folder. This file is important, and this is the one that we will keep under version control with git, so let's go ahead and add it now.

`git add data/adult.data.dvc`  
`git commit -m 'add adult data dvc file'`

And now feel free to take a look at what's in the adult.data.dvc file.

`cat data/adult.data.dvc`  

You will notice an md5 hash value. If you now go back to the .dvc folder you will now see a cache folder has been created. In the cache folder you will see a new folder in there with a name that should match the first couple of digits of the md5 hash value. Navigate into that folder and you'll see that adult.data is now in there.

`cd .dvc/cache/files/md5/{folder name}`  
`cat {file name}`  

This cache is important, it is how DVC will keep track of your different data versions.

To truly take advantage of data version control, we should store our data somewhere else, rather than locally. This way, your coworkers can access the same data. You can store the data in any number of cloud storage options. We may attempt to do this in a future lab.



### Simple Example

We can now modify our toy dataset, and *commit* and *push* those changes. I'm going to now add our `adult.test` data to the folder, and I'll go ahead and add it to DVC.

`dvc add data/adult.test`

Remember now to add the adult.test.dvc and any other changes you've made to your .gitignore files to your git repo. Then, push code changes to Github.

`git add data/.gitignore data/adult.test.dvc`  
`git commit -m 'add adult test data'`  
`git push origin main`  

Now, let's combine adult.data and adult.test. 

`wc -l data/adult.data`  
`cat data/adult.test >> data/adult.data`  
`wc -l data/adult.data`

The adult.data file has been modified. Let's add it to DVC and push the changes.

`dvc add data/adult.data`  
`git add data/adult.data.dvc`  
`git commit -m 'modify adult.data'`  
`git push origin main`   

Now suppose we want to roll back to a previous version of adult.data. We can use git to roll back to the previous version of adult.data.dvc, and then use DVC to checkout the previous version of the data.

`git checkout HEAD~1 data/adult.data.dvc`  
`dvc checkout`  

You can check that the old version of adult.data was checked out by running `wc -l data/adult.data` and noticing the word count matches what it was for the original data.

You can go back to the most recent version again by running `git checkout HEAD data/adult.data.dvc` and `dvc checkout`, but instead, let's keep the previous version and run with it.

`git commit -m 'revert to previous data version'`  
`git push origin main` 



### DVC Pipelines

A typical pipeline for a modeling experiment might have the following stages:

- Import data and create train/validation/test splits   
- Clean data and create features  
- Model training  
- Model evaluation

DVC is capable of keeping track of an entire pipeline like this, though for our purposes we will not include model training and evaluation. 

We've already created the first stage of our pipeline when we added our train and test datasets to DVC. To create the next stage in the data pipeline using DVC, we will create a script that reads in our train and test datasets, makes some changes, and outputs two new datasets for use later. We call this script `create_features.py`. This script loads the data, uses a Pipeline (from sklearn) to transform the data, writes the Pipeline to `pipeline.pkl` and the two transformed data sets to `processed_train_data.csv` and `processed_test_data.csv`. 

We can use DVC to create this new *stage* in our pipeline. DVC will run the code in `create_features.py`. We can create pipelines by specifying them in a `dvc.yaml` file. So, let's create a `dvc.yaml` file in the root directory of our project and add the following pipeline to it.

```
stages:
  prepare:
    cmd: python src/create_features.py
    deps:
      - data/adult.data
      - data/adult.test
      - src/create_features.py
    outs:
      - data/pipeline.pkl
      - data/processed_train_data.csv
      - data/processed_test_data.csv
```

You'll notice that we list out all of the file dependencies that are needed to run our `create_features.py` script, as well as everything that is output from the script. After creating the `dvc.yaml` file we can run `dvc stage list` to see that the *prepare* stage was created.

Now, we can run the pipeline using

`dvc repro`

Once the pipeline is run, we will see several things happen automatically: the outputs of the pipeline will be in the `data` folder, a new file called `dvc.lock` will appear in the root project folder, and the outputs from the pipeline will be added to the `data/.gitignore` file. Take a look at the `dvc.lock` and `data/.gitignore` file before moving on.

From here, you should add the new `dvc.lock`, `dvc.yaml`, and the `data/.gitignore` to the git repo, as well as the create_features.py script. Then commit and push to Github. 

### Parameters

Suppose your pipeline depends on a set of parameters, for example a random seed for doing train/test splits. We can place those parameter values in a **params.yaml** file, and use DVC and git to keep track of changes to these parameters. We would need to change our script to include the yaml file and set the parameters.

Create a params.yaml file that looks like this:
```
features:  
    chi2percentile: 50
    train_path: data/adult.data  
    test_path: data/adult.test
```
Then change the script so that the percentile used for feature selection reads from the yaml file instead of being hard-coded:

`import yaml`  
`params = yaml.safe_load(open("params.yaml"))["features"]`  
`chi2percentile = params["chi2percentile"]`  

and update this line:

`("selector", SelectPercentile(chi2, percentile=chi2percentile)),`

I've included a new version of the script, called `create_features_w_params.py` that you can use for this instead. Now, we'll want to replace the previous pipeline we created in our `dvc.yaml` file by using this new pipeline with parameters. We can simply add the following within our pipeline:

```
params: 
      - features.chi2percentile
      - features.train_path
      - features.test_path
```

Now, run `dvc repro` to see if it works.

When you create pipelines using DVC, you can easily reproduce entire pipelines by running `dvc repro`. For example, if we accidentally changed or deleted one of the output files, such as `data/processed_test_data.csv`, we can simply run `dvc repro` to get it back. In fact, delete both output files and run `dvc repro`. DVC will automatically grab both output files from the cache without needing to rerun the entire script to recreate them.



### DVC Recap

We've shown that we can use DVC to keep track of data versions and pipelines:

- Initialize DVC, similar to a git repository  
- Add datasets to DVC so they are tracked  
- Use git to keep track of the versions of the data through the `{data_name}.dvc` files   
- Roll back to previous versions of data using git  
- Create reproducible pipelines with or without parameters by adding `stages` and using `repro`  

DVC actually does a whole lot more than this, some of which overlaps with other tools. 

# Data Versioning Lab

## Overview

In this lab you will practice using DVC to create data pipelines.

## Goal

The goal in this lab is to become familiar with the importance of keeping track of different versions of your data sets. Although we will use DVC in this lab, we are **not** trying to learn all we can about it. DVC has too much functionality for us to learn in a single lab. 

## Instructions

Using the data that you used in the Experiment Tracking lab, use DVC to create a reproducible pipeline for data preprocessing. 
 
- Put your datasets in a `data/` folder  
- Add your datasets to DVC  

At this point you should create a script, called `preprocessing.py`, saved to the `src/` folder, that does only the data preprocessing stages of the pipeline. Any modeling stages can be kept in a separate script. You may need to first work inside of a notebook to finalize your code. Feel free to just copy and paste any data processing code that you previously used on Lab 2.  

Once your script works, 

- in the root directory of your project, create the `dvc.yaml` file. The `params.yaml` file is optional.  
- run `dvc repro` to create the output files.  
- commit and push changes to Github.  
- remember, do not add your data files to git.

To get credit for this lab, I should be able to see that you've created a pipeline for your project data. There should be: 

- a preprocessing.py file in the `src/` directory,  
- *.dvc files for your datasets in the `data/` directory,     
- dvc.lock and dvc.yaml files in the root directory,    
- an optional params.yaml in the root directory.

### Turning It In

As before, send me a link, in Canvas, to your Github repo. Even though you may have done this already, please do so again to indicate you have finished the lab.

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
import os

# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Load the California housing dataset
housing = fetch_california_housing()

# Convert to pandas DataFrame
housing_df = pd.DataFrame(
    data=np.c_[housing.data, housing.target],
    columns=list(housing.feature_names) + ['target']
)

# Display the first few rows
print("Dataset shape:", housing_df.shape)
housing_df.head()

Dataset shape: (20640, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [2]:
# Save the dataset to a CSV file
housing_df.to_csv('data/california_housing.csv', index=False)
print("Dataset saved to data/california_housing.csv")

Dataset saved to data/california_housing.csv


In [3]:
!dvc add data/california_housing.csv

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/Users/pranavjain/Downloads/MLOps/.dvc/cache/files/md5'|
                                                                                
!
  0%|          |Adding data/california_housing.csv to 0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /Users/pranavjain/Downloa0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 79.08file/s]

To track the changes with git, run:

	git add data/.gitignore data/california_housing.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [4]:
!git add data/california_housing.csv.dvc
!git commit -m "Add California Housing dataset"

[main b3ba355] Add California Housing dataset
 1 file changed, 5 insertions(+)
 create mode 100644 data/california_housing.csv.dvc


In [5]:
# Create a modified version of the dataset
modified_housing_df = housing_df.copy()

# Add a new feature: median_house_value in dollars
modified_housing_df['median_house_value'] = modified_housing_df['target'] * 100000

# Remove some samples (keep only 80% of the data)
modified_housing_df = modified_housing_df.sample(frac=0.8, random_state=42)

# Display the modified dataset
print("Modified dataset shape:", modified_housing_df.shape)
modified_housing_df.head()

Modified dataset shape: (16512, 10)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target,median_house_value
20046,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01,0.47700,47700.0
3024,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46,0.45800,45800.0
15663,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44,5.00001,500001.0
20484,5.7376,17.0,6.163636,1.020202,1705.0,3.444444,34.28,-118.72,2.18600,218600.0
9814,3.7250,34.0,5.492991,1.028037,1063.0,2.483645,36.62,-121.93,2.78000,278000.0


In [6]:
# Save the modified dataset
modified_housing_df.to_csv('data/california_housing.csv', index=False)
print("Modified dataset saved to data/california_housing.csv")

Modified dataset saved to data/california_housing.csv


In [7]:
!dvc add data/california_housing.csv

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/Users/pranavjain/Downloads/MLOps/.dvc/cache/files/md5'|
                                                                                
!
  0%|          |Adding data/california_housing.csv to 0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /Users/pranavjain/Downloa0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 79.89file/s]

To track the changes with git, run:

	git add data/california_housing.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [8]:
!git add data/california_housing.csv.dvc
!git commit -m "Update California Housing dataset with new feature and reduced samples"

[main 041b795] Update California Housing dataset with new feature and reduced samples
 1 file changed, 2 insertions(+), 2 deletions(-)


In [9]:
!dvc status

Data and pipelines are up to date.                                              


In [11]:
# First, let's save our current work
!git add .
!git commit -m "Save current state"

# Now, let's go back to the first version of our dataset
!git checkout HEAD~1 data/california_housing.csv.dvc
!dvc checkout data/california_housing.csv

# Let's verify we have the original dataset
import pandas as pd
reverted_df = pd.read_csv('data/california_housing.csv')
print("Reverted dataset shape:", reverted_df.shape)
reverted_df.head()

[main e9bad71] Save current state
 3 files changed, 820 insertions(+)
 create mode 100644 Lab_3_data_versioning.ipynb
 create mode 100644 data/adult.names
Updated 0 paths from d902bbb
Building workspace index                              |2.00 [00:00,  810entry/s]
Comparing indexes                                    |3.00 [00:00, 4.25kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
Reverted dataset shape: (16512, 10)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target,median_house_value
0,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01,0.47700,47700.0
1,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46,0.45800,45800.0
2,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44,5.00001,500001.0
3,5.7376,17.0,6.163636,1.020202,1705.0,3.444444,34.28,-118.72,2.18600,218600.0
4,3.7250,34.0,5.492991,1.028037,1063.0,2.483645,36.62,-121.93,2.78000,278000.0


In [18]:
!dvc stage list

transform  Outputs data/california_housing_scaled.csv


In [19]:
!dvc repro

'data/california_housing.csv.dvc' didn't change, skipping                       
Running stage 'transform':                                                      
> python src/preprocessing.py data/california_housing.csv data/california_housing_scaled.csv
Transformed data saved to data/california_housing_scaled.csv
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock data/.gitignore

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.
